In [2]:
import time
from nba_api.stats.endpoints import playerprofilev2
from nba_api.stats.endpoints import teamdetails
from nba_api.stats.static import teams
from nba_api.stats.endpoints import CommonTeamRoster
import pandas as pd
import numpy as np

team_id = 1610612747  # Cavs example
seasonPlayed = '2024'

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
def getPlayerImpact(playerId):
  player = playerId
  while True:
    try:
      df = playerprofilev2.PlayerProfileV2(player_id=player, per_mode36='Totals').get_data_frames()[0]
      df = df.tail(1)
      if df['MIN'].iloc[0] < 200:
        return 0
      df['WEIGHTED_RANK_SCORE'] = (
        df['PTS']     * 2.0 +   # Scoring is key
        df['AST']     * 1.8 +   # Playmaking
        df['REB']     * 1.5 +   # Rebounding (OREB + DREB combined)
        df['STL']     * 1.5 +   # Defense
        df['BLK']     * 1.2 +   # Rim protection
        df['TOV']     * -2.0 +  # Turnovers are bad → negative weight
        df['FG_PCT'] * 1.2 +   # Shooting efficiency
        df['FG3_PCT'] * 0.8 +   # 3PT efficiency
        df['FT_PCT']  * 0.6   # Overall efficiency stat
      )
      return np.round(df['WEIGHTED_RANK_SCORE'].values[0] * (36/(df['MIN'].values[0])), 3)
    except Exception as e:
      print(e)
      print('Retrying...(Player Impact)')
      time.sleep(5)

def getPlayers():
  while True:
    try:
      players = CommonTeamRoster(team_id=team_id, season=seasonPlayed).get_data_frames()[0]
      players = players[['PLAYER_ID', 'PLAYER']]
      return players
    except:
      print('Retrying...(Load Players)')
      time.sleep(1.5)


def rankPlayers():
  playerImpacts = []
  players = getPlayers()
  for index, row in players.iterrows():
    id = row['PLAYER_ID']
    name = row['PLAYER']
    playerImpacts.append((name, id, getPlayerImpact(id)))
  playerImpacts.sort(key=lambda x: x[2], reverse=True)
  return pd.DataFrame(playerImpacts, columns=['PLayer Name', 'Id', 'Rank'])

rankPlayers()

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Retrying...(Player Impact)


,PLayer Name,Id,Rank
0,Luka Dončić,1629029,78.653
1,LeBron James,2544,72.300
2,Austin Reaves,1630559,56.688
3,Dalton Knecht,1642261,43.364
4,Jaxson Hayes,1629637,42.675
5,Rui Hachimura,1629060,41.355
6,Shake Milton,1629003,40.568
7,Jarred Vanderbilt,1629020,39.865
8,Jordan Goodwin,1630692,38.174
9,Christian Koloko,1631132,36.641
